## 16S Analysis
This code was run in QIIME2 (v.2019.10) and was used to analyze primate fecal sample communities (stocks) from three primate species and germ free mice inoculated with those stocks. We performed both an analysis of all samples together (stocks + mice) and just the experimental samples (mice). Some analyses were performed in a high performance computing environment, while others were performed locally.

## All
This analysis is of all samples: human, squirrel monkey, and macaque stock communities and the first and last fecal collection timepoints from the mice.

### Import data
Data was imported into QIIME and visualized in a high performance commputing environment. Because of differences in sequencing between stocks and experimental samples, only forward reads were used for this analysis.

In [ ]:
srun --account=b1057 --time=2:00:00 --partition=b1057 --mem=4G --pty bash -l

module load singularity

singularity exec -B /projects/b1057 -B /projects/b1057/kramato \
    -B /projects/p30050  /projects/b1057/qiime2-core-2019-4.simg \
    qiime tools import --type 'SampleData[SequencesWithQuality]' \
    --input-path /projects/b1057/kramato/gf_mouse/micemanifest_single.csv \
    --output-path /projects//b1057/kramato/gf_mouse/mouse-all-single-end-demux.qza \
    --input-format SingleEndFastqManifestPhred33

srun --account=b1057 --time=2:00:00 --partition=b1057 --mem=4G --pty bash -l
module load singularity
singularity exec -B /projects/b1057 -B /projects/b1057/kramato \
    -B /projects/p30050  /projects/b1057/qiime2-core-2019-4.simg \
    qiime tools import --type 'SampleData[SequencesWithQuality]' \
    --input-path /projects/b1057/kramato/gf_mouse/micemanifest_single.csv \
    --output-path /projects//b1057/kramato/gf_mouse/mouse-all-single-end-demux.qza \
    --input-format SingleEndFastqManifestPhred33


### Denoising
Sequences were denoised using the DADA2 algorithm in a high performance commputing environment.

In [ ]:
singularity exec -B /projects/b1057 \
    -B /projects/b1057/kramato /projects/b1057/qiime2-core-2019-4.simg \
    qiime dada2 denoise-single \
    --i-demultiplexed-seqs /projects//b1057/kramato/gf_mouse/mouse-all-single-end-demux.qza \
    --p-trunc-len 150 --p-trim-left 20 --p-max-ee 5 --p-n-threads 8 \
    --o-table /projects/b1057/kramato/gf_mouse/mouse-all-single-table.qza \
    --o-representative-sequences /projects/b1057/kramato/gf_mouse/mouse-all-single-rep-seqs.qza \
    --o-denoising-stats /projects/b1057/kramato/gf_mouse/mouse-all-single-dada2denoising-stats.qza


### Feature table visualization
The results of the DADA2 workflow were visualized locally.

In [ ]:
qiime feature-table tabulate-seqs --i-data mouse-all-single-rep-seqs.qza \
    --o-visualization mouse-all-single-rep-seqs.qzv

qiime feature-table summarize --i-table mouse-all-single-table.qza \
    --m-sample-metadata-file mapping_all.txt \
    --o-visualization mouse-all-single-table.qzv

qiime metadata tabulate --m-input-file mouse-all-single-dada2denoising-stats.qza \
    --o-visualization mouse-all-single-dada2denoising-stats.qzv


### Phylogenetic tree generation
Phylogenetic trees were generated using a standard workflow in QIIME2.

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree --i-sequences mouse-all-single-rep-seqs.qza \
    --o-alignment mouse-all-single-aligned-rep-seqs.qza \
    --o-masked-alignment mouse-all-single-masked-aligned-rep-seqs.qza \
    --o-tree mouse-all-single-unrooted-tree.qza \
    --o-rooted-tree mouse-all-single-rooted-tree.qza


### Filtering tables
Mitochondrial and chloroplast sequences were filtered from the feature tables.

In [ ]:
qiime taxa filter-table --i-table mouse-all-single-table.qza \
    --i-taxonomy mouse-all-single-taxonomy.qza \
    --p-exclude mitochondria,chloroplast \
    --o-filtered-table mouse-all-single-table-nomito-nochloro.qza

qiime feature-table summarize --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-sample-metadata-file mapping_all.txt \
    --o-visualization mouse-all-single-table-nomito-nochloro.qzv


### Alpha diversity
Alpha diversity indices were calculated in QIIME using both the metrics from the standard workflows and Breakaway. Alpha diversity indices were exported for downstream statistical analysis in R.

In [ ]:
qiime diversity alpha-rarefaction --i-table mouse-all-single-table.qza \
    --i-phylogeny mouse-all-single-rooted-tree.qza \
    --o-visualization alpha_rarefaction --p-max-depth 30000

qiime breakaway alpha --i-table mouse-all-single-table-nomito-nochloro.qza \
    --o-alpha-diversity breakaway-richness

qiime tools export --input-path breakaway-richness.qza \
    --output-path breakaway-richness

qiime diversity alpha-phylogenetic --i-table mouse-all-single-table-nomito-nochloro.qza \
    --i-phylogeny mouse-all-single-rooted-tree.qza \
    --o-alpha-diversity pd_diversity --p-metric faith_pd

qiime diversity alpha --i-table mouse-all-single-table-nomito-nochloro.qza \
    --o-alpha-diversity mouse-all-single-shannon --p-metric shannon

qiime tools export --input-path pd_diversity.qza --output-path pd_diversity

qiime tools export --input-path mouse-all-single-shannon.qza --output-path shannon


### Beta diversity
Beta diversity distance matrices were calculated in QIIME. Distance matrices were exported for downstream statistical analysis in R. Samples were rarefied to 10,177 ASVs, which retained all samples.

In [ ]:
qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all_updated_072722.txt --p-where "[Age]='adult'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-noinfant.qza

qiime diversity core-metrics-phylogenetic --i-table mouse-all-single-table-nomito-nochloro-noinfant.qza \
    --i-phylogeny mouse-all-single-rooted-tree.qza --p-sampling-depth 10177 \
    --m-metadata-file mapping_all_updated_072722_noinfant.txt \
    --output-dir core-diversity-even10177-noinfant

qiime tools export --input-path core-diversity-even10177-noinfant/unweighted_unifrac_distance_matrix.qza \
    --output-path core-diversity-even10177-noinfant/unweighted_unifrac_distance_matrix

qiime tools export --input-path core-diversity-even10177-noinfant/weighted_unifrac_distance_matrix.qza \
    --output-path core-diversity-even10177-noinfant/weighted_unifrac_distance_matrix


### Export feature tables
Relative abundance feature tables at the level of phyla, family and genus were created and exported.

In [ ]:
qiime taxa collapse --i-table mouse-all-single-table-nomito-nochloro-noinfant.qza \
    --i-taxonomy mouse-all-single-taxonomy.qza --p-level 2 \
    --o-collapsed-table table-filtered-all-level2-noinfant.qza

qiime taxa collapse --i-table mouse-all-single-table-nomito-nochloro-noinfant.qza \
    --i-taxonomy mouse-all-single-taxonomy.qza --p-level 5 \
    --o-collapsed-table table-filtered-all-level5-noinfant.qza

qiime taxa collapse --i-table mouse-all-single-table-nomito-nochloro-noinfant.qza \
    --i-taxonomy mouse-all-single-taxonomy.qza --p-level 6 \
    --o-collapsed-table table-filtered-all-level6-noinfant.qza

qiime feature-table relative-frequency --i-table table-filtered-all-level2-noinfant.qza \
    --o-relative-frequency-table table-filtered-all-level2-relab-noinfant.qza

qiime feature-table relative-frequency --i-table table-filtered-all-level5-noinfant.qza \
    --o-relative-frequency-table table-filtered-all-level5-relab-noinfant.qza

qiime feature-table relative-frequency --i-table table-filtered-all-level6-noinfant.qza \
    --o-relative-frequency-table table-filtered-all-level6-relab-noinfant.qza

qiime tools export --input-path table-filtered-all-level2-relab-noinfant.qza \
    --output-path exported/level2

qiime tools export --input-path table-filtered-all-level5-relab-noinfant.qza \
    --output-path exported/level5

qiime tools export --input-path table-filtered-all-level6-relab-noinfant.qza \
    --output-path exported/level6

biom convert -i exported/level2/feature-table.biom \
    -o feature-table-level2-relab-noinfant.tsv --to-tsv

biom convert -i exported/level5/feature-table.biom \
    -o feature-table-level5-relab-noinfant.tsv --to-tsv

biom convert -i exported/level6/feature-table.biom \
    -o feature-table-level6-relab-noinfant.tsv --to-tsv

### Core microbiome analysis
We also calculated core microbes shared across stocks and timepoints.

First, we filtered the feature table to include all samples of one stock species plus all samples from either the first or last timepoint.

In [ ]:
qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Stock_species]='human'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-human-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Stock]='human_infant'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-human-infant-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Stock]='human_adult'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-human-adult-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Stock_species]='macaque'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-macque-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Stock]='macaque_infant'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-macaque-infant-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Stock]='macaque_adult'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-macaque-adult-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Stock_species]='SQM'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-sqm-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Stock]='SQM_infant'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-sqm-infant-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Stock]='SQM_adult'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-sqm-adult-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Timepoint]='stock'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-stocks-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Timepoint]='first'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-first-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro.qza \
    --m-metadata-file mapping_all.txt --p-where "[Timepoint]='last'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-last-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro-stocks-only.qza \
    --m-metadata-file mapping_all.txt --p-where "[Age]='adult'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-stocks-adult-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro-first-only.qza \
    --m-metadata-file mapping_all.txt --p-where "[Age]='adult'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-first-adult-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro-last-only.qza \
    --m-metadata-file mapping_all.txt --p-where "[Age]='adult'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-last-adult-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro-stocks-only.qza \
    --m-metadata-file mapping_all.txt --p-where "[Age]='infant'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-stocks-infant-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro-first-only.qza \
    --m-metadata-file mapping_all.txt --p-where "[Age]='infant'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-first-infant-only.qza

qiime feature-table filter-samples --i-table mouse-all-single-table-nomito-nochloro-last-only.qza \
    --m-metadata-file mapping_all.txt --p-where "[Age]='infant'" \
    --o-filtered-table mouse-all-single-table-nomito-nochloro-last-infant-only.qza

We then calculated core features.

In [ ]:
qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-human-only.qza \
    --output-dir humans-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-macque-only.qza \
    --output-dir macaque-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-sqm-only.qza \
    --output-dir sqm-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-stocks-only.qza \
    --output-dir stock-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-first-only.qza \
    --output-dir first-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-last-only.qza \
    --output-dir last-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-human-adult-only.qza \
    --output-dir humans-adult-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-macaque-adult-only.qza \
    --output-dir macaque-adult-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-sqm-adult-only.qza \
    --output-dir sqm-adult-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-stocks-adult-only.qza \
    --output-dir stock-adult-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-first-adult-only.qza \
    --output-dir first-adult-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-last-adult-only.qza \
    --output-dir last-adult-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-human-infant-only.qza \
    --output-dir humans-infant-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-macaque-infant-only.qza \
    --output-dir macaque-infant-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-sqm-infant-only.qza \
    --output-dir sqm-infant-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-stocks-infant-only.qza \
    --output-dir stock-infant-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-first-infant-only.qza \
    --output-dir first-infant-only-core

qiime feature-table core-features --i-table mouse-all-single-table-nomito-nochloro-last-infant-only.qza \
    --output-dir last-infant-only-core

## Experimental
This analysis is of all samples: human, squirrel monkey, and macaque stock communities and the first and last fecal collection timepoints from the mice.

### Import data
Data was imported into QIIME and visualized in a high performance commputing environment. Forward and reverse reads were used for this analysis.

In [ ]:
srun --account=b1057 --time=2:00:00 --partition=b1057 --mem=4G --pty bash -l
        
module load singularity

singularity exec -B /projects/b1057 -B /projects/b1057/kramato \
    -B /projects/p30050 /projects/b1057/qiime2-core-2019-4.simg \
    qiime tools import --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path /projects/b1057/kramato/gf_mouse/micemanifest_exp_only.csv \
    --output-path /projects/b1057/kramato/gf_mouse/mouse-exp-paired-end-demux.qza \
    --input-format PairedEndFastqManifestPhred33

singularity exec -B /projects/b1057 -B /projects/b1057/kramato /projects/b1057/qiime2-core-2019-4.simg \
    qiime demux summarize --i-data /projects/b1057/kramato/gf_mouse/mouse-exp-paired-end-demux.qza \
    --o-visualization /projects//b1057/kramato/gf_mouse/mouse-exp-paired-end-demux.qzv

### Denoising
Sequences were denoised using the DADA2 algorithm in a high performance commputing environment.

In [ ]:
singularity exec -B /projects/b1057 -B /projects/b1057/kramato /projects/b1057/qiime2-core-2019-4.simg \
    qiime dada2 denoise-paired --i-demultiplexed-seqs /projects//b1057/kramato/gf_mouse/mouse-exp-paired-end-demux.qza \
    --p-trunc-len-f 270 --p-trunc-len-r 270 --p-trim-left-f 20 --p-trim-left-r 20 --p-max-ee 5 \
    --p-n-threads 8 --o-table /projects/b1057/kramato/gf_mouse/mouse-exp-table.qza \
    --o-representative-sequences /projects/b1057/kramato/gf_mouse/mouse-exp-rep-seqs.qza \
    --o-denoising-stats /projects/b1057/kramato/gf_mouse/mouse-exp-dada2denoising-stats.qza


### Feature table visualization
The results of the DADA2 workflow were visualized locally.

In [ ]:
qiime feature-table tabulate-seqs --i-data mouse-exp-rep-seqs.qza --o-visualization mouse-exp-paired-rep-seqs.qzv

qiime feature-table summarize --i-table mouse-exp-table.qza --m-sample-metadata-file mapping_exp.txt \
    --o-visualization mouse-exp-paired-table.qzv

qiime metadata tabulate --m-input-file mouse-exp-dada2denoising-stats.qza \
    --o-visualization mouse-exp-paired-dada2denoising-stats.qzv


### Phylogenetic tree generation
Phylogenetic trees were generated using a standard workflow in QIIME2.

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree --i-sequences mouse-exp-rep-seqs.qza \
    --o-alignment mouse-exp-aligned-rep-seqs.qza --o-masked-alignment mouse-exp-masked-aligned-rep-seqs.qza \
    --o-tree mouse-exp-unrooted-tree.qza --o-rooted-tree mouse-exp-paired-rooted-tree.qza

### Filtering tables
Mitochondrial and chloroplast sequences were filtered from the feature tables.

In [ ]:
qiime taxa filter-table --i-table mouse-exp-table.qza --i-taxonomy exp-taxonomy.qza \
    --p-exclude mitochondria,chloroplast --o-filtered-table mouse-exp-paired-table-nomito-nochloro.qza

qiime feature-table summarize --i-table mouse-exp-paired-table-nomito-nochloro.qza \
    --m-sample-metadata-file mapping_exp.txt --o-visualization mouse-exp-paired-table-nomito-nochloro.qzv

### Alpha diversity
Alpha diversity indices were calculated in QIIME using both the metrics from the standard workflows and Breakaway. Alpha diversity indices were exported for downstream statistical analysis in R.

In [ ]:
qiime diversity alpha-rarefaction --i-table mouse-exp-table.qza --i-phylogeny mouse-exp-paired-rooted-tree.qza \
    --o-visualization alpha_rarefaction --p-max-depth 40000

qiime breakaway alpha --i-table mouse-exp-paired-table-nomito-nochloro.qza --o-alpha-diversity breakaway-richness

qiime tools export --input-path breakaway-richness.qza --output-path breakaway-richness

qiime diversity alpha-phylogenetic --i-table mouse-exp-paired-table-nomito-nochloro.qza \
    --i-phylogeny exp-rooted-tree.qza --o-alpha-diversity pd_diversity --p-metric faith_pd

qiime diversity alpha --i-table mouse-exp-paired-table-nomito-nochloro.qza \
    --o-alpha-diversity mouse-exp-paired-shannon --p-metric shannon

qiime tools export --input-path pd_diversity.qza --output-path pd_diversity

qiime tools export --input-path mouse-exp-paired-shannon.qza --output-path shannon


### Beta diversity
Beta diversity distance matrices were calculated in QIIME. Distance matrices were exported for downstream statistical analysis in R. Samples were rarefied to 10,177 ASVs, which retained all samples.

In [ ]:
qiime feature-table filter-samples --i-table mouse-exp-paired-table-nomito-nochloro.qza \
    --m-metadata-file mapping_exp_updated_072722.txt --p-where "[Age]='adult'" \
    --o-filtered-table mouse-exp-paired-table-nomito-nochloro-noinfant.qza

qiime diversity core-metrics-phylogenetic --i-table mouse-exp-paired-table-nomito-nochloro-noinfant.qza \
    --i-phylogeny exp-rooted-tree.qza --p-sampling-depth 6573 \
    --m-metadata-file mapping_exp_updated_072722_noinfant.txt --output-dir core-diversity-even6573-noinfant

qiime tools export --input-path core-diversity-even6573-noinfant/unweighted_unifrac_distance_matrix.qza \
    --output-path core-diversity-even6573-noinfant/unweighted_unifrac_distance_matrix

qiime tools export --input-path core-diversity-even6573-noinfant/weighted_unifrac_distance_matrix.qza \
    --output-path core-diversity-even6573-noinfant/weighted_unifrac_distance_matrix


### Export feature tables
Relative abundance feature tables at the level of phyla, family and genus were created and exported.

In [ ]:
qiime taxa collapse --i-table mouse-exp-paired-table-nomito-nochloro-noinfant.qza --i-taxonomy exp-taxonomy.qza \
    --p-level 2 --o-collapsed-table table-filtered-all-level2-noinfant.qza

qiime taxa collapse --i-table mouse-exp-paired-table-nomito-nochloro-noinfant.qza --i-taxonomy exp-taxonomy.qza \
    --p-level 5 --o-collapsed-table table-filtered-all-level5-noinfant.qza

qiime taxa collapse --i-table mouse-exp-paired-table-nomito-nochloro-noinfant.qza --i-taxonomy exp-taxonomy.qza \
    --p-level 6 --o-collapsed-table table-filtered-all-level6-noinfant.qza

qiime feature-table relative-frequency --i-table table-filtered-all-level2-noinfant.qza \
    --o-relative-frequency-table table-filtered-all-level2-relab-noinfant.qza

qiime feature-table relative-frequency --i-table table-filtered-all-level5-noinfant.qza \
    --o-relative-frequency-table table-filtered-all-level5-relab-noinfant.qza

qiime feature-table relative-frequency --i-table table-filtered-all-level6-noinfant.qza \
    --o-relative-frequency-table table-filtered-all-level6-relab-noinfant.qza

qiime tools export --input-path table-filtered-all-level2-relab-noinfant.qza --output-path exported/level2-noinfant

qiime tools export --input-path table-filtered-all-level5-relab-noinfant.qza --output-path exported/level5-noinfant

qiime tools export --input-path table-filtered-all-level6-relab-noinfant.qza --output-path exported/level6-noinfant

biom convert -i exported/level2-noinfant/feature-table.biom -o feature-table-level2-relab-noinfant.tsv --to-tsv

biom convert -i exported/level5-noinfant/feature-table.biom -o feature-table-level5-relab-noinfant.tsv --to-tsv

biom convert -i exported/level6-noinfant/feature-table.biom -o feature-table-level6-relab-noinfant.tsv --to-tsv


### Absolute abundances
Raw count feature tables were exported to transform to absolute abundances using 16S rt-qPCR data. Transformed tables were then reimported into QIIME, filtered, and exported for downstream statistical analysis in R.

In [ ]:
qiime feature-table filter-samples --i-table mouse-exp-table.qza --m-metadata-file mapping_exp_updated_072722.txt \
    --p-where "[Age]='adult'" --o-filtered-table mouse-exp-paired-table-noinfant.qza

qiime tools export --input-path  mouse-exp-paired-table-noinfant.qza --output-path mouse-exp-paired-table-noinfant

biom convert -i mouse-exp-paired-table-noinfant/feature-table.biom -o mouse-exp-paired-table-noinfant.txt --to-tsv

biom convert -i inferred_abundance_table_noinfant_t.txt -o inferred_abundance_table_noinfant.biom --to-hdf5 \
    --table-type="OTU table"

qiime tools import --input-path inferred_abundance_table_noinfant.biom --type 'FeatureTable[Frequency]' \
    --input-format BIOMV210Format --output-path inferred_abundance_table_noinfant.qza

qiime taxa filter-table --i-table inferred_abundance_table_noinfant.qza --i-taxonomy exp-taxonomy.qza \
    --p-exclude mitochondria,chloroplast --o-filtered-table inferred_abundance_table-nomito-nochloro-noinfant.qza

qiime tools export --input-path inferred_abundance_table-nomito-nochloro-noinfant.qza \
    --output-path exported/inferred-noinfant

qiime taxa collapse --i-table inferred_abundance_table-nomito-nochloro-noinfant.qza \
    --i-taxonomy exp-taxonomy.qza --p-level 2 --o-collapsed-table inferred-table-filtered-level2-noinfant.qza

qiime taxa collapse --i-table inferred_abundance_table-nomito-nochloro-noinfant.qza \
    --i-taxonomy exp-taxonomy.qza --p-level 5 --o-collapsed-table inferred-table-filtered-level5-noinfant.qza

qiime taxa collapse --i-table inferred_abundance_table-nomito-nochloro-noinfant.qza \
    --i-taxonomy exp-taxonomy.qza --p-level 6 --o-collapsed-table inferred-table-filtered-level6-noinfant.qza

qiime tools export --input-path inferred-table-filtered-level2-noinfant.qza \
    --output-path exported/inferred-level2-noinfant

qiime tools export --input-path inferred-table-filtered-level5-noinfant.qza \
    --output-path exported/inferred-level5-noinfant

qiime tools export --input-path inferred-table-filtered-level6-noinfant.qza \
    --output-path exported/inferred-level6-noinfant

biom convert -i exported/inferred-level2-noinfant/feature-table.biom \
    -o inferred-feature-table-level2-relab-noinfant.tsv --to-tsv

biom convert -i exported/inferred-level5-noinfant/feature-table.biom \
    -o inferred-feature-table-level5-relab-noinfant.tsv --to-tsv

biom convert -i exported/inferred-level6-noinfant/feature-table.biom \
    -o inferred-feature-table-level6-relab-noinfant.tsv --to-tsv